# 4.2 - Procesos paralelos


![parallel](images/parallel.png)

$$$$

### Multiprocessing

Veamos en primer lugar [multiprocessing](https://docs.python.org/es/3.9/library/multiprocessing.html). Es una librería de Python que nos permite manejar hilos y procesos. La diferencia entre hilo y proceso es que un hilo ocurre dentro del espacio de memoria de un programa y un proceso es una copia completa del programa, por esta razón, los hilos son rápidos de crear y destruir además de que consumen poca memoria y los procesos son lentos de crear y destruir además de que requieren clonar el espacio de memoria del programa en otro lugar de la RAM, y esto es lento. Ejemplos de esto serían, subrutinas que recogen mensajes de un puerto de comunicaciones y los usan para actuar sobre emails almacenados en un servidor, desde el punto de vista del servidor, el cliente de correo sólo necesita usar el servidor durante un corto plazo de tiempo, porque envía un mensaje al servidor donde le indica lo que el usuario desea hacer, saber si hay mensajes nuevos, borrar un correo, moverlo... El servidor abre un hilo para atender a ese usuario y el hilo sólo vive mientras dure la conexión del usuario, una vez el usuario ha terminado el cliente de correo desconecta hasta nueva acción. Este proceso que he descrito es rápido, ocurre en milisegundos y generalmente se resuelve con hilos porque es más ligero para el sistema operativo y su vida media es especialmente corta, además de que el sistema podrá aceptar ciento o miles de conexiones por segundo y será ligero, rápido y eficiente en esta tarea.

La tendencia actual entre los desarrolladores es hacer una aplicaciones que sean rápidas en un sólo hilo y luego escalar a tantas instancias como sea necesario para cubrir nuestros objetivos de aprovechamiento, estos servidores pueden atender en un sólo proceso a miles o decena de miles de conexiones.

Si queremos realizar un programa que aproveche las diferentes CPUs y pueda realizar múltiples tareas a la vez tenemos muchos mecanismos para llevar esta tarea a cabo. Dependiendo del uso que se quiera dar probablemente queramos usar hilos o procesos, es aquí donde querremos escribir nuestro código con hilos o procesos.

**Hola Mundo**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
def cuadrado(x):
    return x**2

In [3]:
datos=[i for i in range(10000000)]

In [4]:
%%time

seq=[cuadrado(x) for x in datos]

seq[:10]

CPU times: user 2.26 s, sys: 55.1 ms, total: 2.32 s
Wall time: 2.32 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [5]:
%%time

map(cuadrado, datos)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


In [7]:
%%time

a=list(map(cuadrado, datos))

CPU times: user 2.13 s, sys: 41.8 ms, total: 2.17 s
Wall time: 2.17 s


In [8]:
import multiprocessing as mp

In [9]:
mp.get_start_method()   # movida del Mac M1, para otros no hace falta

'spawn'

In [10]:
# movida del mac M1, para otros no hace falta

from multiprocessing import get_context

In [11]:
mp.cpu_count()

8

In [12]:
%%time

#pool=mp.Pool(mp.cpu_count())   # grupo con todos los cores

pool=get_context('fork').Pool(6)  # grupo con 6 cores

res=pool.map(cuadrado, datos)
pool.close()

res[:10]

CPU times: user 633 ms, sys: 350 ms, total: 983 ms
Wall time: 1.4 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [13]:
pool

<multiprocessing.pool.Pool state=CLOSE pool_size=6>

**multiprocessing asíncrono**

`map` consume su iterable convirtiendo el iterable en una lista, dividiéndolo en fragmentos y enviando esos fragmentos a los procesos de trabajo en el Pool. Dividir el iterable en fragmentos funciona mejor que pasar cada elemento en el iterable entre procesos un elemento a la vez, especialmente si el iterable es grande. Sin embargo, convertir el iterable en una lista para dividirlo puede tener un costo de memoria muy alto, ya que la lista completa deberá mantenerse en la memoria.

`imap`/`map_async` no convierte el iterable que le da en una lista, ni lo divide en trozos. Itera sobre el elemento de uno en uno y los envia a un proceso de trabajo distinto. Esto significa que no se toma el golpe de memoria de convertir todo el iterable en una lista, pero también que el rendimiento es más lento para los iterables grandes, debido a la falta de fragmentación. Esto se puede mitigar aumentando el valor predeterminado de 1 en el `chunksize`. Otra gran diferencia de `imap` es que puede comenzar a recibir resultados de los trabajadores tan pronto como estén listos, en lugar de tener que esperar a que todos terminen. 




In [14]:
%%time

pool=get_context('fork').Pool(6)  # grupo con 6 cores

res=pool.map_async(cuadrado, datos).get()
pool.close()

res[:10]

CPU times: user 522 ms, sys: 400 ms, total: 922 ms
Wall time: 1.09 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

```python
%%time
pool=mp.Pool(mp.cpu_count())   

for x in pool.imap(cuadrado, datos):
    print(x)
    
pool.close()
```

$$$$

$$$$

## Joblib

![joblib](images/joblib.svg)

$$$$

$$$$


[Joblib](https://joblib.readthedocs.io/en/latest/) es una librería de Python que también nos permite paralelizar un programa. En este caso a través de procesos, lo cuál implica, como vimos antes, cierto tiempo para construir el Pool. Lo usaremos principalmente para realizar un bucle sobre una función.

Veamos el Hola Mundo.

**Hola Mundo**

In [17]:
%pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [18]:
from joblib import Parallel, delayed

In [24]:
%%time

funcion=lambda x: x**0.5   # raiz cuadrada

lst=Parallel(n_jobs=6, verbose=True)(delayed(funcion)(i**2) for i in range(100000))

lst[:10]

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  46 tasks      | elapsed:    0.4s


CPU times: user 1.13 s, sys: 388 ms, total: 1.52 s
Wall time: 1.41 s


[Parallel(n_jobs=6)]: Done 87878 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done 100000 out of 100000 | elapsed:    1.4s finished


[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

### Ejemplo ESPN

Volvamos de nuevo al ejemplo de scrapeo de la págine de ESPN. Usaremos joblib para realizar una extracción en paralelo de la información.

In [25]:
from selenium import webdriver

import time

import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [/Users/iudh/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache


In [31]:
url='https://www.espn.com/soccer/competitions'


driver=webdriver.Chrome(PATH)
driver.get(url)
    
time.sleep(2)

aceptar=driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
aceptar.click()

time.sleep(4)

equipos=driver.find_element_by_css_selector('#fittPageContainer > div.page-container.cf > div > div.layout__column.layout__column--1 > div > div:nth-child(3) > div:nth-child(1) > div > div:nth-child(5) > div > section > div > div > span:nth-child(2) > a')
equipos.click()


time.sleep(2)

equipos_stats_urls=driver.find_elements_by_css_selector('a.AnchorLink')

equipos_stats_urls=[e.get_attribute('href') for e in equipos_stats_urls 
                    if 'team/stats' in e.get_attribute('href')]


equipos_stats_urls


['https://www.espn.com/soccer/team/stats/_/id/598/1-fc-union-berlin',
 'https://www.espn.com/soccer/team/stats/_/id/2506/arminia-bielefeld',
 'https://www.espn.com/soccer/team/stats/_/id/131/bayer-leverkusen',
 'https://www.espn.com/soccer/team/stats/_/id/132/bayern-munich',
 'https://www.espn.com/soccer/team/stats/_/id/124/borussia-dortmund',
 'https://www.espn.com/soccer/team/stats/_/id/268/borussia-monchengladbach',
 'https://www.espn.com/soccer/team/stats/_/id/125/eintracht-frankfurt',
 'https://www.espn.com/soccer/team/stats/_/id/3841/fc-augsburg',
 'https://www.espn.com/soccer/team/stats/_/id/122/fc-cologne',
 'https://www.espn.com/soccer/team/stats/_/id/129/hertha-berlin',
 'https://www.espn.com/soccer/team/stats/_/id/2950/mainz',
 'https://www.espn.com/soccer/team/stats/_/id/11420/rb-leipzig',
 'https://www.espn.com/soccer/team/stats/_/id/126/sc-freiburg',
 'https://www.espn.com/soccer/team/stats/_/id/3070/spvgg-greuther-furth',
 'https://www.espn.com/soccer/team/stats/_/id/791

In [32]:
driver.quit()

In [34]:
def extraer(url):
    
    driver=webdriver.Chrome(PATH)
    driver.get(url)

    time.sleep(2)

    aceptar=driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
    aceptar.click()

    time.sleep(2)

    disciplina=driver.find_element_by_xpath('//*[@id="fittPageContainer"]/div[2]/div[5]/div/div/section/div/div[2]/nav/ul/li[2]/a')
    disciplina.click()

    time.sleep(1)

    tabla=driver.find_element_by_tag_name('tbody')

    filas=tabla.find_elements_by_tag_name('tr')

    datos=[[e.text for e in f.find_elements_by_tag_name('td')]+[url.split('/')[-1]] for f in filas]
    
    

    cabeceras=driver.find_element_by_tag_name('thead')

    cabeceras=[c.text for c in cabeceras.find_elements_by_tag_name('th')]+['TEAM']
    
    driver.quit()
    
    return pd.DataFrame(datos, columns=cabeceras)

In [35]:
%%time

lst_df=Parallel(n_jobs=6, verbose=True)(delayed(extraer)(url) for url in equipos_stats_urls)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.

/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:26: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:28: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:26: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:28: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fv/zc

/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:18: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:20: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:22: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:26: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:28: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, valu

/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:18: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:20: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:22: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:18: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:20: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, valu

CPU times: user 105 ms, sys: 39.3 ms, total: 144 ms
Wall time: 46.5 s


/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:26: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_7398/2205445825.py:28: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
[Parallel(n_jobs=6)]: Done  18 out of  18 | elapsed:   46.5s finished


In [36]:
len(lst_df)

18

In [37]:
len(equipos_stats_urls)

18

In [38]:
lst_df[0].head()

,RK,NAME,P,YC,RC,PTS,TEAM
0,1,Rani Khedira,32,6,0,6,1-fc-union-berlin
1,,Robin Knoche,33,6,0,6,1-fc-union-berlin
2,3,Grischa Prömel,29,5,0,5,1-fc-union-berlin
3,,Paul Jaeckel,24,5,0,5,1-fc-union-berlin
4,,Timo Baumgartl,25,5,0,5,1-fc-union-berlin


**Tip:** https://pypi.org/project/tqdm/

In [39]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [46]:
from tqdm.notebook import tqdm    # from tqdm import tqdm   # para .py

In [41]:
lst=[1, 23, 45, 65, 6778, 8756, 32]


for e in lst:
    print(e**2)

1
529
2025
4225
45941284
76667536
1024


In [42]:
for e in tqdm(lst):
    print(e**2)

  0%|          | 0/7 [00:00<?, ?it/s]

1
529
2025
4225
45941284
76667536
1024


In [47]:
%%time

funcion=lambda x: x**0.5   # raiz cuadrada

lst=Parallel(n_jobs=6, verbose=True)(delayed(funcion)(i**2) for i in tqdm(range(1000000)))

lst[:10]

  0%|          | 0/1000000 [00:00<?, ?it/s]

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done 100 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 524276 tasks      | elapsed:    2.4s


CPU times: user 2.98 s, sys: 68.2 ms, total: 3.05 s
Wall time: 3.04 s


[Parallel(n_jobs=6)]: Done 1000000 out of 1000000 | elapsed:    3.0s finished


[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]